In [1]:
#importing useful modules
import pickle, math, pandas as pd
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import numpy as np 
from numpy.linalg import norm

In [2]:
# dimen = [50, 100, 200, 300]
# mod_name = ['cbow', 'fasttext', 'glove', 'sg']
#storing dimensions and model names
dimen = [50, 100]
mod_name = ['cbow']

In [3]:
#glove_mod_creation creates the glove model since there are only text files.
#run this only once to create glove models
# def glove_mod_creation(dimen, mod_name):
#     path = 'dataset\\hi'+'\\' +str(dimen)+'\\'+str(mod_name)+'\\hi-d'+str(dimen)+'-m2-'+str(mod_name)+'.txt'
#     mod_path = 'dataset\\hi'+'\\' +str(dimen)+'\\glove\\hi-d'+str(dimen)+'-m2-'+str(mod_name)+'.model'
#     file = open(path, 'r', encoding = 'utf8', errors = 'replace')
#     model_dict = {}
#     i = 1
#     while True:
#         line = file.readline()
#         if not line:
#             break
#         lineval = line.split()
#         model_dict[lineval[0]] = [float(lineval[i]) for i in range(1, len(lineval))]           
#     with open(mod_path, 'wb') as f:
#         pickle.dump(model_dict, f)

# for d_val in dimen:
#     for m_name in mod_name:
#         if m_name == 'glove':
#             glove_mod_creation(d_val, m_name)

In [4]:
#given ground truth and threshold*10
ground_truth = [6.8,7.0,7.6,7.6,8.0,6.0,6.2,7.6,6.4,5.8,6.6,5.8,7.6,4.6,7.0,5.2,6.2,7.6,8.0,7.0,0.0,8.2,7.0,7.8,
                1.0,7.8,9.0,6.4,5.8,7.4,7.8,6.6,6.6,8.4,1.4,0.4,7.0,7.2,1.8,6.8,8.4,6.6,7.4,1.4,6.2,0.2,2.0,8.6,
                9.2,8.4,8.2,6.6,2.6,8.0,7.6,6.8,6.2,5.2,7.0,6.4,7.0,7.4,6.2,7.0,4.2]
threshold = [4, 5, 6, 7, 8]

In [5]:
#function to calculate cosine similarity
def dot(v1, v2):
    ans = 0.0
    for i in range(len(v1)):
        ans += v1[i]*v2[i]
    return ans

def norm(v1):
    ans = 0.0
    for val in v1:
        ans += val**2
    return math.sqrt(ans)

def cos_sim(vec1, vec2):
    return (dot(vec1, vec2)/(norm(vec1)*norm(vec2)))*10

In [6]:
#creating_vec function takes one model and returns bit vector for hindi similarity dataset word pairs
def creating_vec(model, mod_name, threshold):
    vec = []
    path = 'dataset\\Wordsimilarity_datasets\\iiith_wordsim\\hindi.txt'
    word_sim = open(path, 'r', encoding = 'utf8')
    lines = word_sim.readlines()
    for line in lines:
        words = line.split(',')
        if mod_name != 'glove':
            vec1 = model.wv[words[0]]
            vec2 = model.wv[words[1]]
        else:
            vec1 = model[words[0]]
            vec2 = model[words[1]]
        cosine = cos_sim(vec1, vec2)
        if cosine >= threshold:
            vec.append(1)
        else:
            vec.append(0)
    return vec

In [7]:
#process function helps to load the model in RAM
def process(dimen, mod_name):
    path = 'dataset\\hi'+'\\' +str(dimen)+'\\'+str(mod_name)+'\\hi-d'+str(dimen)+'-m2-'+str(mod_name)+'.model'
    if mod_name != 'glove': 
        mod_load = Word2Vec.load(path)
        return mod_load
    else:
        with open(path, 'rb') as f:
            mod_load = pickle.load(f)
        return mod_load

In [8]:
#truth_vec function returns bit vector according to the threshold value
def truth_vec(thres, truth_val):
    vecc = []
    for item in truth_val:
        if item >= thres:
            vecc.append(1)
        else:
            vecc.append(0)
    return vecc

In [9]:
#dfcreation function creates dataframe and returns it as mentioned in forum
def dfcreation(model, mod_name, accuracy, threshold):
    df = pd.DataFrame(columns=['Word1', 'Word2', 'similarity_score', 'ground_truth_similarity', 'label'])
    path = 'dataset\\Wordsimilarity_datasets\\iiith_wordsim\\hindi.txt'
    word_sim = open(path, 'r', encoding = 'utf8')
    lines = word_sim.readlines()
    for words in lines:
        splitwords = words.split(',')
        c_score = cosine(model, mod_name, splitwords[0], splitwords[1])
        df.loc[len(df.index)] = [splitwords[0], splitwords[1], c_score, float(splitwords[2]), 1 if c_score >= threshold else 0]
    df.loc[len(df.index)] = ['Total Accuracy --> '+str(accuracy), '', '', '', '']
    return df


In [10]:
#cosine function calculates cosine score between two words wrt given model
def cosine(model, mod_name, word1, word2):
    if mod_name != 'glove':
        vec1 = model.wv[word1]
        vec2 = model.wv[word2]
    else:
        vec1 = model[word1]
        vec2 = model[word2]
    return cos_sim(vec1, vec2)

In [11]:
#main program which calls each function and calculate the result
for d_val in dimen:
    for m_name in mod_name:
        model = process(d_val, m_name)
        accuracy = 0.0
        for thres in threshold:
            vect = creating_vec(model, m_name, thres)
            truth_val = truth_vec(thres, ground_truth)
            result = list(a^b for a,b in zip(vect, truth_val))
            accuracy = ((65-sum(result))/65)*100
            dataf = dfcreation(model, m_name, accuracy, thres)
            dataf.to_csv('Answer_1_output\\Q1_'+str(d_val)+'_'+str(m_name)+'_similarity_'+str(thres)+'.csv', index=False)
            print('Accuracy calculated successfully for dimension', d_val, ', model ', m_name, 'and threshold ', thres)


Accuracy calculated successfully for dimension 50 , model  cbow and threshold  4
Accuracy calculated successfully for dimension 50 , model  cbow and threshold  5
Accuracy calculated successfully for dimension 50 , model  cbow and threshold  6
Accuracy calculated successfully for dimension 50 , model  cbow and threshold  7
Accuracy calculated successfully for dimension 50 , model  cbow and threshold  8
Accuracy calculated successfully for dimension 100 , model  cbow and threshold  4
Accuracy calculated successfully for dimension 100 , model  cbow and threshold  5
Accuracy calculated successfully for dimension 100 , model  cbow and threshold  6
Accuracy calculated successfully for dimension 100 , model  cbow and threshold  7
Accuracy calculated successfully for dimension 100 , model  cbow and threshold  8
